# Structured Text Insights Extraction Demo

This notebook demonstrates the **Structured Text Insights Flow** using the Bloomberg Financial News dataset. 

## What You'll Learn
- How to use the structured insights flow for comprehensive text analysis
- Extract summaries, keywords, entities, and sentiment from financial news
- Analyze and visualize results across large datasets
- Extend the flow with custom blocks for domain-specific analysis

## Flow Capabilities
The structured insights flow performs **4 key analyses** on any text:
1. **📝 Summary**: Concise 2-3 sentence summaries
2. **🔑 Keywords**: Top 10 most important terms
3. **🏷️ Entities**: Named entities (people, organizations, locations)
4. **😊 Sentiment**: Emotional tone analysis (positive/negative/neutral)

All results are combined into a **structured JSON output** for easy processing and analysis.

## Setup and Installation

In [1]:
%load_ext autoreload
%autoreload 2

# pip install sdg_hub[examples]

In [2]:
import json
import random
import warnings

from datasets import load_dataset
import nest_asyncio

from sdg_hub import Flow, FlowRegistry

warnings.filterwarnings('ignore')

# Required for async execution in notebooks
nest_asyncio.apply()

/Users/shiv/workspace/sdg_hub/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Flow Discovery and Loading

SDG Hub automatically discovers all available flows. Let's find our structured insights flow:

In [3]:
# Auto-discover all available flows
FlowRegistry.discover_flows()

# List all flows
flows = FlowRegistry.list_flows()

[15:18:35] INFO     Discovered 5 flows                                                              ]8;id=396155;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/registry.py\registry.py]8;;\:]8;id=934608;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/registry.py#113\113]8;;\

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┓
┃ ID               ┃ Name                  ┃ Author               ┃ Tags                  ┃ Description           ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━┩
│ epic-jade-656    │ Extractive Summary    │ SDG Hub Contributors │ knowledge-tuning,     │ Generate extractive   │
│                  │ Knowledge Tuning      │                      │ document-internaliza… │ summary from the      │
│                  │ Dataset Generation    │                      │ question-generation,  │ input document. Each  │
│                  │ Flow                  │                      │ knowledge-extractive… │ document is first     │
│                  │                       │                      │ qa-pairs,             │ converted into list   │
│                  │                       │                      │ extractive-summaries  │ of knowledge segments │
│                  │                       │                      │                       │ for creating          │
│                  │                       │                      │                       │ extractive summary    │
│                  │                       │                      │                       │ and then annotated    │
│                  │                       │                      │                       │ with context,         │
│                  │                       │                      │                       │ relationship and      │
│                  │                       │                      │                       │ relevance. This is    │
│                  │                       │                      │                       │ then converted into   │
│                  │                       │                      │                       │ Question-Answer       │
│                  │                       │                      │                       │ pairs.                │
│ green-clay-812   │ Structured Text       │ SDG Hub Contributors │ text-analysis,        │ Multi-step pipeline   │
│                  │ Insights Extraction   │                      │ summarization, nlp,   │ for extracting        │
│                  │ Flow                  │                      │ structured-output,    │ structured insights   │
│                  │                       │                      │ insights,             │ from text including   │
│                  │                       │                      │ sentiment-analysis,   │ summary, keywords,    │
│                  │                       │                      │ entity-extraction,    │ entities, and         │
│                  │                       │                      │ keyword-extraction    │ sentiment analysis    │
│                  │                       │                      │                       │ combined into a JSON  │
│                  │                       │                      │                       │ output                │
│ heavy-heart-77   │ Key Facts Knowledge   │ SDG Hub Contributors │ knowledge-tuning,     │ Generates training    │
│                  │ Tuning Dataset        │                      │ document-internaliza… │ datasets for          │
│                  │ Generation Flow       │                      │ question-generation,  │ knowledge tuning by   │
│                  │                       │                      │ knowledge-extraction, │ creating diverse      │
│                  │                       │                      │ qa-pairs,             │ question-answer pairs │
│                  │                       │                      │ document-processing,  │ from documents. Uses  │
│                  │                       │                      │ educational,          │ three summarization   │
│                  │                       │            

In [4]:
# Search for text analysis flows
text_flows = FlowRegistry.search_flows(tag="text-analysis")
print(f"Text analysis flows: {text_flows}")

# Load our structured insights flow
flow_id = "green-clay-812" 
flow_path = FlowRegistry.get_flow_path(flow_id)
flow = Flow.from_yaml(flow_path)

print(f"\n✅ Loaded flow: {flow_id}") 

flow.print_info()

Text analysis flows: [{'id': 'green-clay-812', 'name': 'Structured Text Insights Extraction Flow'}]


[15:18:35] INFO     Loading flow from:                                                                  ]8;id=257475;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=582566;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/base.py#172\172]8;;\
                    /Users/shiv/workspace/sdg_hub/src/sdg_hub/flows/text_analysis/structured_insights/f            
                    low.yaml                                                                                       


✅ Loaded flow: green-clay-812


╭─────────────────────────────────────────────── Flow Information ────────────────────────────────────────────────╮
│ Structured Text Insights Extraction Flow Flow                                                                   │
│ ├── Metadata                                                                                                    │
│ │   ├── Version: 1.0.0                                                                                          │
│ │   ├── Author: SDG Hub Contributors                                                                            │
│ │   └── Description: Multi-step pipeline for extracting structured insights from text including summary,        │
│ │       keywords, entities, and sentiment analysis combined into a JSON output                                  │
│ └── Blocks (13 total)                                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Block Details ─────────────────────────────────────────────────╮
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┓ │
│ ┃ Block Name                 ┃ Type               ┃ Input Cols                      ┃ Output Cols             ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━┩ │
│ │ build_summary_prompt       │ PromptBuilderBlock │ ['text']                        │ ['summary_prompt']      │ │
│ │ generate_summary           │ LLMChatBlock       │ ['summary_prompt']              │ ['raw_summary']         │ │
│ │ parse_summary              │ TextParserBlock    │ ['raw_summary']                 │ ['summary']             │ │
│ │ build_keywords_prompt      │ PromptBuilderBlock │ ['text']                        │ ['keywords_prompt']     │ │
│ │ generate_keywords          │ LLMChatBlock       │ ['keywords_prompt']             │ ['raw_keywords']        │ │
│ │ parse_keywords             │ TextParserBlock    │ ['raw_keywords']                │ ['keywords']            │ │
│ │ build_entities_prompt      │ PromptBuilderBlock │ ['text']                        │ ['entities_prompt']     │ │
│ │ generate_entities          │ LLMChatBlock       │ ['entities_prompt']             │ ['raw_entities']        │ │
│ │ parse_entities             │ TextParserBlock    │ ['raw_entities']                │ ['entities']            │ │
│ │ build_sentiment_prompt     │ PromptBuilderBlock │ ['text']                        │ ['sentiment_prompt']    │ │
│ │ generate_sentiment         │ LLMChatBlock       │ ['sentiment_prompt']            │ ['raw_sentiment']       │ │
│ │ parse_sentiment            │ TextParserBlock    │ ['raw_sentiment']               │ ['sentiment']           │ │
│ │ create_structured_insights │ JSONStructureBlock │ ['summary', 'keywords',         │ ['structured_insights'] │ │
│ │                            │                    │ 'entities', 'sentiment']        │                         │ │
│ └────────────────────────────┴────────────────────┴─────────────────────────────────┴─────────────────────────┘ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## 2. Model Configuration

The flow supports multiple LLM models. Let's configure it:

In [5]:
# Check recommended models
print("Default model:", flow.get_default_model())
print("Model recommendations:", flow.get_model_recommendations())

Default model: openai/gpt-oss-120b
Model recommendations: {'default': 'openai/gpt-oss-120b', 'compatible': ['meta-llama/Llama-3.3-70B-Instruct', 'microsoft/phi-4', 'mistralai/Mixtral-8x7B-Instruct-v0.1'], 'experimental': ['gpt-4o']}


In [6]:
# Configure the flow to use a specific model
# Option 1: Use a local vLLM server
flow.set_model_config(
    model="hosted_vllm/openai/gpt-oss-120b",
    api_base="http://localhost:8201/v1",
    api_key="EMPTY",
    # this only works with models which support reasoning_effort
    # if your model does not support it, you can remove this parameter
    extra_body={"reasoning_effort": "low"}
)

# Option 2: Use OpenAI (requires API key)
# flow.set_model_config(
#     model="gpt-4o-mini",
#     api_key="your-openai-api-key"
# )

# Option 3: Use Anthropic Claude (requires API key)
# flow.set_model_config(
#     model="anthropic/claude-3-haiku",
#     api_key="your-anthropic-api-key"
# )

print("✅ Model configuration ready")

           INFO     Auto-detected 4 LLM blocks for configuration: ['generate_entities',                 ]8;id=291396;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=440878;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/base.py#864\864]8;;\
                    'generate_keywords', 'generate_sentiment', 'generate_summary']                                 

[15:18:35] INFO     Loaded LLM client for model 'hosted_vllm/openai/gpt-oss-120b'              ]8;id=306589;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/blocks/llm/client_manager.py\client_manager.py]8;;\:]8;id=239361;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/blocks/llm/client_manager.py#61\61]8;;\

[15:18:35] INFO     Initialized LLMChatBlock 'generate_summary' with model                    ]8;id=242992;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/blocks/llm/llm_chat_block.py\llm_chat_block.py]8;;\:]8;id=372401;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/blocks/llm/llm_chat_block.py#265\265]8;;\
                    'hosted_vllm/openai/gpt-oss-120b'                                                              

           INFO     Loaded LLM client for model 'hosted_vllm/openai/gpt-oss-120b'              ]8;id=747070;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/blocks/llm/client_manager.py\client_manager.py]8;;\:]8;id=154749;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/blocks/llm/client_manager.py#61\61]8;;\

           INFO     Initialized LLMChatBlock 'generate_keywords' with model                   ]8;id=638474;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/blocks/llm/llm_chat_block.py\llm_chat_block.py]8;;\:]8;id=48241;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/blocks/llm/llm_chat_block.py#265\265]8;;\
                    'hosted_vllm/openai/gpt-oss-120b'                                                              

...

           INFO     Loaded LLM client for model 'hosted_vllm/openai/gpt-oss-120b'              ]8;id=597388;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/blocks/llm/client_manager.py\client_manager.py]8;;\:]8;id=252803;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/blocks/llm/client_manager.py#61\61]8;;\

           INFO     Initialized LLMChatBlock 'generate_sentiment' with model                  ]8;id=618912;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/blocks/llm/llm_chat_block.py\llm_chat_block.py]8;;\:]8;id=566524;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/blocks/llm/llm_chat_block.py#265\265]8;;\
                    'hosted_vllm/openai/gpt-oss-120b'                                                              

           INFO     Successfully configured 4 LLM blocks with: model:                                   ]8;id=563346;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=483653;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/base.py#903\903]8;;\
                    'hosted_vllm/openai/gpt-oss-120b', api_base: 'http://localhost:8201/v1', api_key:              
                    EMPTY, extra_body: {'reasoning_effort': 'low'}                                                 

           INFO     Configured blocks: ['generate_entities', 'generate_keywords', 'generate_sentiment', ]8;id=406501;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=806391;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/base.py#906\906]8;;\
                    'generate_summary']                                                                            

✅ Model configuration ready


## 3. Dataset Loading and Exploration

We'll use the **Bloomberg Financial News dataset** - 447k financial news articles from 2006-2013:

In [7]:
# Load the Bloomberg Financial News dataset
print("Loading Bloomberg Financial News dataset...")
dataset = load_dataset("danidanou/Bloomberg_Financial_News", split="train")

print(f"📊 Dataset size: {len(dataset):,} articles")
print(f"📅 Columns: {dataset.column_names}")
print(f"💾 Dataset features: {dataset.features}")

Loading Bloomberg Financial News dataset...
📊 Dataset size: 446,762 articles
📅 Columns: ['Headline', 'Journalists', 'Date', 'Link', 'Article']
💾 Dataset features: {'Headline': Value(dtype='string', id=None), 'Journalists': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'Date': Value(dtype='timestamp[ns]', id=None), 'Link': Value(dtype='string', id=None), 'Article': Value(dtype='string', id=None)}


In [8]:
# Explore the dataset structure
sample = dataset[0]
print("=== Sample Article ===")
print(f"Headline: {sample['Headline']}")
print(f"Date: {sample['Date']}")
print(f"Journalists: {sample['Journalists']}")
print(f"Article length: {len(sample['Article'])} characters")
print(f"Article preview: {sample['Article'][:300]}...")

=== Sample Article ===
Headline: Ivory Coast Keeps Cocoa Export Tax Below 22%, Document Shows
Date: 2011-10-06 15:14:20
Journalists: ['Baudelaire Mieu']
Article length: 2530 characters
Article preview: Export taxes on cocoa beans from Ivory Coast , the world’s biggest producer of the chocolate ingredient, won’t exceed 22 percent of the international price this season, meeting a commitment to the International Monetary Fund , according to a finance ministry document. In the 2008-9 season taxes aver...


In [9]:
# Select a small sample for demonstration (start with 50 articles)
# For production, you can process thousands of articles
sample_size = 50
demo_dataset = dataset.shuffle(seed=42).select(range(sample_size))

print(f"📝 Demo dataset prepared: {len(demo_dataset)} articles")
print(f"📊 Average article length: {sum(len(article['Article']) for article in demo_dataset) / len(demo_dataset):.0f} characters")

📝 Demo dataset prepared: 50 articles
📊 Average article length: 2397 characters


In [10]:
# Discover what dataset schema is expected by the flow

schema_dataset = flow.get_dataset_schema() 
print(f"Required columns: {schema_dataset.column_names}")
print(f"Schema: {schema_dataset.features}")

Required columns: ['text']
Schema: {'text': Value(dtype='string', id=None)}


In [11]:
# The flow expects a 'text' column, so we'll use rename the 'Article' column to 'text'
demo_dataset = demo_dataset.rename_column("Article", "text")

## 4. Running the Structured Insights Flow

Now let's extract structured insights from our financial news articles:

In [12]:
# Generate structured insights
print("🚀 Running structured insights extraction...")
print("⏱️ This may take a few minutes depending on your model setup...")

# Run the flow
results = flow.generate(demo_dataset)

print("✅ Processing complete!")
print(f"📊 Generated insights for {len(results)} articles")
print(f"📋 Result columns: {results.column_names}")

🚀 Running structured insights extraction...
⏱️ This may take a few minutes depending on your model setup...


[15:18:37] INFO     Starting flow 'Structured Text Insights Extraction Flow' v1.0.0 with 50 samples     ]8;id=339763;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=168649;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/base.py#515\515]8;;\
                    across 13 blocks                                                                               

           INFO     Executing block 1/13: build_summary_prompt (PromptBuilderBlock)                     ]8;id=36224;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=387258;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/base.py#670\670]8;;\

╭───────────────────────────────────────────── build_summary_prompt ──────────────────────────────────────────────╮
│ 📊 Processing Input Data                                                                                        │
│ Block Type: PromptBuilderBlock                                                                                  │
│ Input Rows: 50                                                                                                  │
│ Input Columns: 5                                                                                                │
│ Column Names: Headline, Journalists, Date, Link, text                                                           │
│ Expected Output Columns: summary_prompt                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────── build_summary_prompt - Complete ────────────────────────────────────────╮
│ ✅ Processing Complete                                                                                          │
│ Rows: 50 → 50                                                                                                   │
│ Columns: 5 → 6                                                                                                  │
│ 🟢 Added: summary_prompt                                                                                        │
│ 📋 Final Columns: Date, Headline, Journalists, Link, summary_prompt, text                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────── create_structured_insights - Complete ─────────────────────────────────────╮
│ ✅ Processing Complete                                                                                          │
│ Rows: 50 → 50                                                                                                   │
│ Columns: 17 → 18                                                                                                │
│ 🟢 Added: structured_insights                                                                                   │
│ 📋 Final Columns: Date, Headline, Journalists, Link, entities, entities_prompt, keywords, keywords_prompt,      │
│ raw_entities, raw_keywords, raw_sentiment, raw_summary, sentiment, sentiment_prompt, structured_insights,       │
│ summary, summary_prompt, text                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

           INFO     Block 'create_structured_insights' completed successfully: 50 samples, 18 columns   ]8;id=718570;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=46399;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/base.py#732\732]8;;\

╭────────────────────────────── Structured Text Insights Extraction Flow - Complete ──────────────────────────────╮
│                                        Flow Execution Summary                                                   │
│ ┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓           │
│ ┃ Block Name           ┃ Type            ┃   Duration ┃     Rows     ┃     Columns     ┃   Status   ┃           │
│ ┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩           │
│ │ build_summary_prompt │ PromptBuilderB… │      0.01s │   50 → 50    │       +1        │     ✓      │           │
│ │ generate_summary     │ LLMChatBlock    │      1.81s │   50 → 50    │       +1        │     ✓      │           │
│ │ parse_summary        │ TextParserBlock │      0.02s │   50 → 50    │       +1        │     ✓      │           │
│ │ build_keywords_prom… │ PromptBuilderB… │      0.01s │   50 → 50    │       +1        │     ✓      │           │
│ │ generate_keywords    │ LLMChatBlock    │      1.14s │   50 → 50    │       +1        │     ✓      │           │
│ │ parse_keywords       │ TextParserBlock │      0.02s │   50 → 50    │       +1        │     ✓      │           │
│ │ build_entities_prom… │ PromptBuilderB… │      0.01s │   50 → 50    │       +1        │     ✓      │           │
│ │ generate_entities    │ LLMChatBlock    │      4.05s │   50 → 50    │       +1        │     ✓      │           │
│ │ parse_entities       │ TextParserBlock │      0.03s │   50 → 50    │       +1        │     ✓      │           │
│ │ build_sentiment_pro… │ PromptBuilderB… │      0.02s │   50 → 50    │       +1        │     ✓      │           │
│ │ generate_sentiment   │ LLMChatBlock    │      0.60s │   50 → 50    │       +1        │     ✓      │           │
│ │ parse_sentiment      │ TextParserBlock │      0.04s │   50 → 50    │       +1        │     ✓      │           │
│ │ create_structured_i… │ JSONStructureB… │      0.02s │   50 → 50    │       +1        │     ✓      │           │
│ ├──────────────────────┼─────────────────┼────────────┼──────────────┼─────────────────┼────────────┤           │
│ │ TOTAL                │ 13 blocks       │      7.77s │   50 final   │    18 final     │   13/13    │           │
│ └──────────────────────┴─────────────────┴────────────┴──────────────┴─────────────────┴────────────┘           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

           INFO     Flow 'Structured Text Insights Extraction Flow' completed successfully: 50 final    ]8;id=770959;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=830464;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/base.py#620\620]8;;\
                    samples, 18 final columns                                                                      

✅ Processing complete!
📊 Generated insights for 50 articles
📋 Result columns: ['Headline', 'Journalists', 'Date', 'Link', 'text', 'summary_prompt', 'raw_summary', 'summary', 'keywords_prompt', 'raw_keywords', 'keywords', 'entities_prompt', 'raw_entities', 'entities', 'sentiment_prompt', 'raw_sentiment', 'sentiment', 'structured_insights']


In [13]:
# Display a sample result
sample_result = results[random.randint(0, len(results) - 1)]

print("=== First Article Analysis ===")
print(f"📰 Original headline: {dataset[0]['Headline']}")
print(f"📅 Date: {dataset[0]['Date']}")
print(f"✍️ Journalists: {dataset[0]['Journalists']}")
print(f"📄 Article length: {len(sample_result['text'])} characters")
print()

# Parse and display the structured insights
insights = json.loads(sample_result["structured_insights"])
print("🔍 EXTRACTED INSIGHTS:")
print(json.dumps(insights, indent=2, ensure_ascii=False))

=== First Article Analysis ===
📰 Original headline: Ivory Coast Keeps Cocoa Export Tax Below 22%, Document Shows
📅 Date: 2011-10-06 15:14:20
✍️ Journalists: ['Baudelaire Mieu']
📄 Article length: 6138 characters

🔍 EXTRACTED INSIGHTS:
{
  "summary": "As of July 13 2012, Dreyfus Cash Management Funds reported daily and weekly yields ranging from 0.00% to 0.16% across various share classes (institutional, investor, administrative, etc.) with total assets per fund ranging from about $376 K to $27.9 million. The listed funds include cash management, government, treasury, municipal, tax‑exempt, and amount‑free municipal shares, each identified by CUSIP and maturity days. For more details or assistance, contact the Dreyfus Institutional Client Administrator at (800) 346‑3621 or Tom Haje at (609) 279‑4146.",
  "keywords": "Dreyfus Cash Management, Institutional Shares, Yield%, Maturity, CUSIP, Treasury Cash Management, Municipal Cash Management, Administrative Shares, Bloomberg, Millrate",
  "

## 5. Dynamic Flow Extension: Adding Stock Ticker Extraction

Now we'll demonstrate SDG Hub's **dynamic flow modification** capabilities. Instead of creating separate flow files, we can extend flows at runtime by adding custom processing blocks using existing SDG Hub components.

### What We'll Add:
We'll extend our structured insights flow to extract **stock ticker symbols** from financial news articles. This is perfect for Bloomberg financial news analysis!

### Approach:
We'll use three existing SDG Hub blocks:
1. **PromptBuilderBlock** - Create a prompt to extract stock tickers
2. **LLMChatBlock** - Process the extraction using the LLM
3. **TextParserBlock** - Parse the output to a clean list

Let's see how to modify flows at runtime!

In [14]:
# We'll modify the existing flow by adding our ticker extraction blocks
# First, let's examine the current flow structure
flow.print_info()

╭─────────────────────────────────────────────── Flow Information ────────────────────────────────────────────────╮
│ Structured Text Insights Extraction Flow Flow                                                                   │
│ ├── Metadata                                                                                                    │
│ │   ├── Version: 1.0.0                                                                                          │
│ │   ├── Author: SDG Hub Contributors                                                                            │
│ │   └── Description: Multi-step pipeline for extracting structured insights from text including summary,        │
│ │       keywords, entities, and sentiment analysis combined into a JSON output                                  │
│ └── Blocks (13 total)                                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Block Details ─────────────────────────────────────────────────╮
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┓ │
│ ┃ Block Name                 ┃ Type               ┃ Input Cols                      ┃ Output Cols             ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━┩ │
│ │ build_summary_prompt       │ PromptBuilderBlock │ ['text']                        │ ['summary_prompt']      │ │
│ │ generate_summary           │ LLMChatBlock       │ ['summary_prompt']              │ ['raw_summary']         │ │
│ │ parse_summary              │ TextParserBlock    │ ['raw_summary']                 │ ['summary']             │ │
│ │ build_keywords_prompt      │ PromptBuilderBlock │ ['text']                        │ ['keywords_prompt']     │ │
│ │ generate_keywords          │ LLMChatBlock       │ ['keywords_prompt']             │ ['raw_keywords']        │ │
│ │ parse_keywords             │ TextParserBlock    │ ['raw_keywords']                │ ['keywords']            │ │
│ │ build_entities_prompt      │ PromptBuilderBlock │ ['text']                        │ ['entities_prompt']     │ │
│ │ generate_entities          │ LLMChatBlock       │ ['entities_prompt']             │ ['raw_entities']        │ │
│ │ parse_entities             │ TextParserBlock    │ ['raw_entities']                │ ['entities']            │ │
│ │ build_sentiment_prompt     │ PromptBuilderBlock │ ['text']                        │ ['sentiment_prompt']    │ │
│ │ generate_sentiment         │ LLMChatBlock       │ ['sentiment_prompt']            │ ['raw_sentiment']       │ │
│ │ parse_sentiment            │ TextParserBlock    │ ['raw_sentiment']               │ ['sentiment']           │ │
│ │ create_structured_insights │ JSONStructureBlock │ ['summary', 'keywords',         │ ['structured_insights'] │ │
│ │                            │                    │ 'entities', 'sentiment']        │                         │ │
│ └────────────────────────────┴────────────────────┴─────────────────────────────────┴─────────────────────────┘ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [15]:
# Import the blocks we need
from sdg_hub.core.blocks.llm import LLMChatBlock, PromptBuilderBlock, TextParserBlock
from sdg_hub.core.blocks.transform import JSONStructureBlock

# Step 1: Add stock ticker extraction blocks to the flow
print("🚀 Adding stock ticker extraction blocks to the flow...")

# Create the stock ticker extraction blocks
ticker_prompt_block = PromptBuilderBlock(
    block_name="stock_ticker_prompt",
    input_cols=["text"],
    output_cols=["ticker_prompt"],
    prompt_config_path="extract_stock_tickers.yaml"
)

ticker_llm_block = LLMChatBlock(
    block_name="extract_stock_tickers",
    input_cols=["ticker_prompt"],
    output_cols=["raw_stock_tickers"],
    max_tokens=512,
    temperature=0.1  # Low temperature for more consistent extraction
)

ticker_parser_block = TextParserBlock(
    block_name="parse_stock_tickers",
    input_cols=["raw_stock_tickers"],
    output_cols=["stock_tickers"],
    start_tags=["[STOCK_TICKERS]"],
    end_tags=["[/STOCK_TICKERS]"]
)

print("✅ Created ticker extraction blocks:")
print(f"  1. {ticker_prompt_block.block_name} - Builds extraction prompt")
print(f"  2. {ticker_llm_block.block_name} - Extracts tickers via LLM")
print(f"  3. {ticker_parser_block.block_name} - Parses LLM output")

# Step 2: Update the JSONStructureBlock to include stock tickers
print("🔧 Updating JSON structure to include stock ticker field...")

# Create a new JSONStructureBlock configuration that includes our new stock_tickers field
enhanced_json_block = JSONStructureBlock(
    block_name="create_enhanced_structured_insights",
    input_cols=["summary", "keywords", "entities", "sentiment", "stock_tickers"],
    output_cols=["enhanced_structured_insights"]
)

print("✅ Enhanced JSON structure will include:")
print("  📝 summary - Article summary")
print("  🔑 keywords - Important keywords")
print("  🏷️ entities - Named entities")
print("  😊 sentiment - Emotional tone")
print("  📈 stock_tickers - Stock ticker symbols (NEW!)")

🚀 Adding stock ticker extraction blocks to the flow...
✅ Created ticker extraction blocks:
  1. stock_ticker_prompt - Builds extraction prompt
  2. extract_stock_tickers - Extracts tickers via LLM
  3. parse_stock_tickers - Parses LLM output
🔧 Updating JSON structure to include stock ticker field...
✅ Enhanced JSON structure will include:
  📝 summary - Article summary
  🔑 keywords - Important keywords
  🏷️ entities - Named entities
  😊 sentiment - Emotional tone
  📈 stock_tickers - Stock ticker symbols (NEW!)


In [16]:

# Remove the original JSONStructureBlock (if it exists in your flow/blocks list)
# (Assume we are not using a flow object here, just not using the old block.)

# Add the new blocks to a list for the enhanced pipeline
ticker_blocks = [
    ticker_prompt_block,
    ticker_llm_block,
    ticker_parser_block,
    enhanced_json_block
]

flow.blocks.pop()
flow.blocks.extend(ticker_blocks)
flow.print_info()


╭─────────────────────────────────────────────── Flow Information ────────────────────────────────────────────────╮
│ Structured Text Insights Extraction Flow Flow                                                                   │
│ ├── Metadata                                                                                                    │
│ │   ├── Version: 1.0.0                                                                                          │
│ │   ├── Author: SDG Hub Contributors                                                                            │
│ │   └── Description: Multi-step pipeline for extracting structured insights from text including summary,        │
│ │       keywords, entities, and sentiment analysis combined into a JSON output                                  │
│ └── Blocks (16 total)                                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Block Details ─────────────────────────────────────────────────╮
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓ │
│ ┃ Block Name                  ┃ Type               ┃ Input Cols                 ┃ Output Cols                 ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩ │
│ │ build_summary_prompt        │ PromptBuilderBlock │ ['text']                   │ ['summary_prompt']          │ │
│ │ generate_summary            │ LLMChatBlock       │ ['summary_prompt']         │ ['raw_summary']             │ │
│ │ parse_summary               │ TextParserBlock    │ ['raw_summary']            │ ['summary']                 │ │
│ │ build_keywords_prompt       │ PromptBuilderBlock │ ['text']                   │ ['keywords_prompt']         │ │
│ │ generate_keywords           │ LLMChatBlock       │ ['keywords_prompt']        │ ['raw_keywords']            │ │
│ │ parse_keywords              │ TextParserBlock    │ ['raw_keywords']           │ ['keywords']                │ │
│ │ build_entities_prompt       │ PromptBuilderBlock │ ['text']                   │ ['entities_prompt']         │ │
│ │ generate_entities           │ LLMChatBlock       │ ['entities_prompt']        │ ['raw_entities']            │ │
│ │ parse_entities              │ TextParserBlock    │ ['raw_entities']           │ ['entities']                │ │
│ │ build_sentiment_prompt      │ PromptBuilderBlock │ ['text']                   │ ['sentiment_prompt']        │ │
│ │ generate_sentiment          │ LLMChatBlock       │ ['sentiment_prompt']       │ ['raw_sentiment']           │ │
│ │ parse_sentiment             │ TextParserBlock    │ ['raw_sentiment']          │ ['sentiment']               │ │
│ │ stock_ticker_prompt         │ PromptBuilderBlock │ ['text']                   │ ['ticker_prompt']           │ │
│ │ extract_stock_tickers       │ LLMChatBlock       │ ['ticker_prompt']          │ ['raw_stock_tickers']       │ │
│ │ parse_stock_tickers         │ TextParserBlock    │ ['raw_stock_tickers']      │ ['stock_tickers']           │ │
│ │ create_enhanced_structured… │ JSONStructureBlock │ ['summary', 'keywords',    │ ['enhanced_structured_insi… │ │
│ │                             │                    │ 'entities', 'sentiment',   │                             │ │
│ │                             │                    │ 'stock_tickers']           │                             │ │
│ └─────────────────────────────┴────────────────────┴────────────────────────────┴─────────────────────────────┘ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [17]:
# Configure the new LLM blocks with our model settings
flow.set_model_config(
    model="hosted_vllm/openai/gpt-oss-120b",
    api_base="http://localhost:8201/v1", 
    api_key="EMPTY",
    # this only works with models which support reasoning_effort
    # if your model does not support it, you can remove this parameter
    extra_body={"reasoning_effort": "low"}
)

print("\n🎯 Ready to run enhanced flow with stock ticker extraction!")

           INFO     Auto-detected 5 LLM blocks for configuration: ['extract_stock_tickers',             ]8;id=30480;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=527682;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/base.py#864\864]8;;\
                    'generate_entities', 'generate_keywords', 'generate_sentiment', 'generate_summary']            

[15:18:44] INFO     Loaded LLM client for model 'hosted_vllm/openai/gpt-oss-120b'              ]8;id=563206;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/blocks/llm/client_manager.py\client_manager.py]8;;\:]8;id=815211;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/blocks/llm/client_manager.py#61\61]8;;\

           INFO     Initialized LLMChatBlock 'generate_summary' with model                    ]8;id=554149;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/blocks/llm/llm_chat_block.py\llm_chat_block.py]8;;\:]8;id=276166;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/blocks/llm/llm_chat_block.py#265\265]8;;\
                    'hosted_vllm/openai/gpt-oss-120b'                                                              

           INFO     Loaded LLM client for model 'hosted_vllm/openai/gpt-oss-120b'              ]8;id=301702;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/blocks/llm/client_manager.py\client_manager.py]8;;\:]8;id=923055;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/blocks/llm/client_manager.py#61\61]8;;\

           INFO     Initialized LLMChatBlock 'generate_keywords' with model                   ]8;id=522047;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/blocks/llm/llm_chat_block.py\llm_chat_block.py]8;;\:]8;id=427985;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/blocks/llm/llm_chat_block.py#265\265]8;;\
                    'hosted_vllm/openai/gpt-oss-120b'                                                              

...

           INFO     Loaded LLM client for model 'hosted_vllm/openai/gpt-oss-120b'              ]8;id=117139;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/blocks/llm/client_manager.py\client_manager.py]8;;\:]8;id=158377;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/blocks/llm/client_manager.py#61\61]8;;\

           INFO     Initialized LLMChatBlock 'extract_stock_tickers' with model               ]8;id=789761;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/blocks/llm/llm_chat_block.py\llm_chat_block.py]8;;\:]8;id=726964;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/blocks/llm/llm_chat_block.py#265\265]8;;\
                    'hosted_vllm/openai/gpt-oss-120b'                                                              

           INFO     Successfully configured 5 LLM blocks with: model:                                   ]8;id=544094;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=568704;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/base.py#903\903]8;;\
                    'hosted_vllm/openai/gpt-oss-120b', api_base: 'http://localhost:8201/v1', api_key:              
                    EMPTY, extra_body: {'reasoning_effort': 'low'}                                                 

           INFO     Configured blocks: ['extract_stock_tickers', 'generate_entities',                   ]8;id=274655;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=511695;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/base.py#906\906]8;;\
                    'generate_keywords', 'generate_sentiment', 'generate_summary']                                 


🎯 Ready to run enhanced flow with stock ticker extraction!


In [18]:
# Generate structured insights
print("🚀 Running structured insights extraction...")
print("⏱️ This may take a few minutes depending on your model setup...")

# Run the flow
results2 = flow.generate(demo_dataset)

print("✅ Processing complete!")
print(f"📊 Generated insights for {len(results2)} articles")
print(f"📋 Result columns: {results2.column_names}")

🚀 Running structured insights extraction...
⏱️ This may take a few minutes depending on your model setup...


           INFO     Starting flow 'Structured Text Insights Extraction Flow' v1.0.0 with 50 samples     ]8;id=426521;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=671132;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/base.py#515\515]8;;\
                    across 16 blocks                                                                               

           INFO     Executing block 1/16: build_summary_prompt (PromptBuilderBlock)                     ]8;id=106077;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=129853;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/base.py#670\670]8;;\

╭───────────────────────────────────────────── build_summary_prompt ──────────────────────────────────────────────╮
│ 📊 Processing Input Data                                                                                        │
│ Block Type: PromptBuilderBlock                                                                                  │
│ Input Rows: 50                                                                                                  │
│ Input Columns: 5                                                                                                │
│ Column Names: Headline, Journalists, Date, Link, text                                                           │
│ Expected Output Columns: summary_prompt                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────── build_summary_prompt - Complete ────────────────────────────────────────╮
│ ✅ Processing Complete                                                                                          │
│ Rows: 50 → 50                                                                                                   │
│ Columns: 5 → 6                                                                                                  │
│ 🟢 Added: summary_prompt                                                                                        │
│ 📋 Final Columns: Date, Headline, Journalists, Link, summary_prompt, text                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────── create_enhanced_structured_insights - Complete ─────────────────────────────────╮
│ ✅ Processing Complete                                                                                          │
│ Rows: 50 → 50                                                                                                   │
│ Columns: 20 → 21                                                                                                │
│ 🟢 Added: enhanced_structured_insights                                                                          │
│ 📋 Final Columns: Date, Headline, Journalists, Link, enhanced_structured_insights, entities, entities_prompt,   │
│ keywords, keywords_prompt, raw_entities, raw_keywords, raw_sentiment, raw_stock_tickers, raw_summary,           │
│ sentiment, sentiment_prompt, stock_tickers, summary, summary_prompt, text, ticker_prompt                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

           INFO     Block 'create_enhanced_structured_insights' completed successfully: 50 samples, 21  ]8;id=84689;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=752100;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/base.py#732\732]8;;\
                    columns                                                                                        

╭────────────────────────────── Structured Text Insights Extraction Flow - Complete ──────────────────────────────╮
│                                        Flow Execution Summary                                                   │
│ ┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓           │
│ ┃ Block Name           ┃ Type            ┃   Duration ┃     Rows     ┃     Columns     ┃   Status   ┃           │
│ ┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩           │
│ │ build_summary_prompt │ PromptBuilderB… │      0.00s │   50 → 50    │       +1        │     ✓      │           │
│ │ generate_summary     │ LLMChatBlock    │      1.60s │   50 → 50    │       +1        │     ✓      │           │
│ │ parse_summary        │ TextParserBlock │      0.02s │   50 → 50    │       +1        │     ✓      │           │
│ │ build_keywords_prom… │ PromptBuilderB… │      0.01s │   50 → 50    │       +1        │     ✓      │           │
│ │ generate_keywords    │ LLMChatBlock    │      1.18s │   50 → 50    │       +1        │     ✓      │           │
│ │ parse_keywords       │ TextParserBlock │      0.03s │   50 → 50    │       +1        │     ✓      │           │
│ │ build_entities_prom… │ PromptBuilderB… │      0.01s │   50 → 50    │       +1        │     ✓      │           │
│ │ generate_entities    │ LLMChatBlock    │      4.00s │   50 → 50    │       +1        │     ✓      │           │
│ │ parse_entities       │ TextParserBlock │      0.03s │   50 → 50    │       +1        │     ✓      │           │
│ │ build_sentiment_pro… │ PromptBuilderB… │      0.02s │   50 → 50    │       +1        │     ✓      │           │
│ │ generate_sentiment   │ LLMChatBlock    │      0.69s │   50 → 50    │       +1        │     ✓      │           │
│ │ parse_sentiment      │ TextParserBlock │      0.04s │   50 → 50    │       +1        │     ✓      │           │
│ │ stock_ticker_prompt  │ PromptBuilderB… │      0.02s │   50 → 50    │       +1        │     ✓      │           │
│ │ extract_stock_ticke… │ LLMChatBlock    │     18.27s │   50 → 50    │       +1        │     ✓      │           │
│ │ parse_stock_tickers  │ TextParserBlock │      0.04s │   50 → 50    │       +1        │     ✓      │           │
│ │ create_enhanced_str… │ JSONStructureB… │      0.02s │   50 → 50    │       +1        │     ✓      │           │
│ ├──────────────────────┼─────────────────┼────────────┼──────────────┼─────────────────┼────────────┤           │
│ │ TOTAL                │ 16 blocks       │     25.98s │   50 final   │    21 final     │   16/16    │           │
│ └──────────────────────┴─────────────────┴────────────┴──────────────┴─────────────────┴────────────┘           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

[15:19:11] INFO     Flow 'Structured Text Insights Extraction Flow' completed successfully: 50 final    ]8;id=85409;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=612014;file:///Users/shiv/workspace/sdg_hub/src/sdg_hub/core/flow/base.py#620\620]8;;\
                    samples, 21 final columns                                                                      

✅ Processing complete!
📊 Generated insights for 50 articles
📋 Result columns: ['Headline', 'Journalists', 'Date', 'Link', 'text', 'summary_prompt', 'raw_summary', 'summary', 'keywords_prompt', 'raw_keywords', 'keywords', 'entities_prompt', 'raw_entities', 'entities', 'sentiment_prompt', 'raw_sentiment', 'sentiment', 'ticker_prompt', 'raw_stock_tickers', 'stock_tickers', 'enhanced_structured_insights']


In [19]:
# Display a sample result
sample_result2 = results2[random.randint(0, len(results2) - 1)]

print("=== First Article Analysis ===")
print(f"📰 Original headline: {dataset[0]['Headline']}")
print(f"📅 Date: {dataset[0]['Date']}")
print(f"✍️ Journalists: {dataset[0]['Journalists']}")
print(f"📄 Article length: {len(sample_result2['text'])} characters")
print()

# Parse and display the structured insights
insights2 = json.loads(sample_result2["enhanced_structured_insights"])
print("🔍 EXTRACTED INSIGHTS:")
print(json.dumps(insights2, indent=2, ensure_ascii=False))

=== First Article Analysis ===
📰 Original headline: Ivory Coast Keeps Cocoa Export Tax Below 22%, Document Shows
📅 Date: 2011-10-06 15:14:20
✍️ Journalists: ['Baudelaire Mieu']
📄 Article length: 7649 characters

🔍 EXTRACTED INSIGHTS:
{
  "summary": "Arthur Laurents, the acclaimed New York playwright and screenwriter behind landmark Broadway hits such as “West Side Story” and “Gyvy,” died at 93 from pneumonia complications. Over a seven‑decade career he collaborated with major figures like Leonard Bernstein, Jerome Robbins, and Stephen Sondheim, wrote notable films including “The Way We Were,” and directed celebrated productions and revivals, earning a reputation for both artistic brilliance and a blunt, volatile personality.",
  "keywords": "Arthur Laurents, West Side Story, Gypsy, Leonard Bernstein, Stephen Sondheim, Broadway revivals, Gypsy Rose Lee, Hollywood blacklist, Tony Award, La Cage aux Folles",
  "entities": "{\n  \"people\": [\n    \"Arthur Laurents\",\n    \"Leonard Bernst

## Next Steps

### 🧪 **Experiment Further**
1. **Scale up**: Process 100+ articles to see larger patterns
2. **Time analysis**: Filter by date ranges to see trends over time
3. **Model comparison**: Try different LLMs and compare results
4. **Custom prompts**: Modify the prompt templates for your domain

### 🔧 **Customize for Your Use Case**
1. **Domain adaptation**: Modify prompts for your specific industry
2. **Additional insights**: Add blocks for topic classification, urgency scoring, etc.
3. **Output format**: Customize JSON structure for your applications
4. **Quality filters**: Add validation and quality checks

### 🚀 Build Your Own Model
- Leverage the generated structured insights as high-quality training data for your own machine learning models.
- Fine-tune LLMs or train classifiers to automate similar analyses at scale.
- Refer to Training Hub (https://github.com/Red-Hat-AI-Innovation-Team/training_hub) to setup your own training pipeline.

### 📚 **Learn More**
- Explore other SDG Hub flows in the repository
- Check the documentation for advanced configuration options
- Join the community for questions and contributions